In [1]:
import os

In [2]:
%pwd

'f:\\ML-Project\\research'

In [3]:
os.chdir('../')
%pwd

'f:\\ML-Project'

In [4]:
from mlproject.constants import *
from mlproject.utils.common import read_yaml,create_directories,save_json

[2025-01-10 17:30:22,327 : INFO : __init__ : Logger has been set up successfully!]


In [4]:
from dataclasses import dataclass
from pathlib import Path
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import joblib
import json
from mlproject.constants import *
from mlproject.utils.common import read_yaml, create_directories, save_json

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_path: Path
    target_column: str


class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH,
        schema_filepath=SCHEMA_FILE_PATH,
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config["artifacts_root"]])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config["model_evaluation"]
        params = self.params["HistGradientBoostingClassifier"]
        schema = self.schema["TARGET_COLUMN"]

        create_directories([config["root_dir"]])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=Path(config["root_dir"]),
            test_data_path=Path(config["test_data_path"]),
            model_path=Path(config["model_path"]),
            all_params=params,
            metric_file_path=Path(config["metric_file_path"]),
            target_column=schema["name"],
        )
        return model_evaluation_config


class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def eval_metrics(self, actual, pred):
        accuracy = accuracy_score(actual, pred)
        precision = precision_score(actual, pred, average="weighted")
        recall = recall_score(actual, pred, average="weighted")
        f1 = f1_score(actual, pred, average="weighted")
        return accuracy, precision, recall, f1

    def save_results(self):
        # Load test data and model
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        # Split features and target
        test_x = test_data.drop(columns=[self.config.target_column])
        test_y = test_data[self.config.target_column]

        # Predict using the loaded model
        predicted_qualities = model.predict(test_x)

        # Compute metrics
        accuracy, precision, recall, f1 = self.eval_metrics(test_y, predicted_qualities)

        # Save metrics as JSON
        scores = {
            "accuracy": accuracy,
            "precision": precision,
            "recall": recall,
            "f1": f1,
        }
        save_json(path=self.config.metric_file_path, data=scores)


# Main Execution
try:
    config_manager = ConfigurationManager()
    model_evaluation_config = config_manager.get_model_evaluation_config()

    model_evaluator = ModelEvaluation(config=model_evaluation_config)
    model_evaluator.save_results()

except Exception as e:
    print(f"Error: {e}")
    raise e


[2025-01-10 17:50:16,222 : INFO : __init__ : Logger has been set up successfully!]
[2025-01-10 17:50:16,345 : INFO : common : yaml file: config\config.yaml loaded successfully]
[2025-01-10 17:50:16,345 : INFO : common : yaml file: params.yaml loaded successfully]
[2025-01-10 17:50:16,345 : INFO : common : yaml file: schema.yaml loaded successfully]
[2025-01-10 17:50:16,345 : INFO : common : created directory at: artifacts]
[2025-01-10 17:50:16,345 : INFO : common : created directory at: artifacts/model_evaluation]
[2025-01-10 17:50:16,995 : INFO : common : json file saved at: artifacts\model_evaluation\metrics.json]


f:\ProgramFiles\anaconda3\envs\mlproject\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
from dataclasses import dataclass
from pathlib import Path    

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_path: Path
    traget_colunm: str
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import joblib
import json
from pathlib import Path


class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
       config = self.config.model_evaluation
       params = self.params.HistGradientBoostingClassifier
       schema = self.schema.TARGET_COLUMN

       create_directories([config.root_dir])
       model_evaluation_config = ModelEvaluationConfig(
            root_dir = Path(config.root_dir),
            test_data_path = Path(config.test_data_path),
            model_path = Path(config.model_path),
            all_params = params,
            metric_file_path = Path(config.metric_file_path),
            tragetcolunm = schema.name
         )
       
       return model_evaluation_config
    

class ModelEvaluation:
    def __init__(self, config):
        self.config = config

    def eval_metrics(self, actual, pred):
        accuracy = accuracy_score(actual, pred)
        precision = precision_score(actual, pred, average="weighted")
        recall = recall_score(actual, pred, average="weighted")
        f1 = f1_score(actual, pred, average="weighted")
        return accuracy,precision,recall,f1

    def save_results(self):
        # Load test data and model
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        # Split features and target
        test_x = test_data.drop(columns=[self.config.target_column])
        test_y = test_data[self.config.target_column]

        # Predict using the loaded model
        predicted_qualities = model.predict(test_x)
        # Compute metrics
        (accuracy,precision,recall,f1) = self.eval_metrics(test_y, predicted_qualities)
        # Save metrics as JSON
        scores = {"accuracy": accuracy, "precision": precision, "recall": recall,"f1":f1}
        save_json(path=Path(self.config.metric_file_name), data=scores)
        


In [6]:
print(config.model_evaluation)

NameError: name 'config' is not defined

In [7]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    
except Exception as e:
    raise e

[2025-01-10 17:30:59,153 : INFO : common : yaml file: config\config.yaml loaded successfully]
[2025-01-10 17:30:59,164 : INFO : common : yaml file: params.yaml loaded successfully]


[2025-01-10 17:30:59,171 : INFO : common : yaml file: schema.yaml loaded successfully]
[2025-01-10 17:30:59,171 : INFO : common : created directory at: artifacts]
[2025-01-10 17:30:59,171 : INFO : common : created directory at: artifacts/model_evaluation]


TypeError: ModelEvaluationConfig.__init__() got an unexpected keyword argument 'tragetcolunm'